# Install required Python modules

*Note:* This step may take a while to run the first time.

In [1]:
# Install uv for simplified package management. (It is not necessary to understand this step.)
!curl -LsSf https://astral.sh/uv/install.sh | sh

downloading uv 0.8.3 x86_64-unknown-linux-gnu
no checksums to verify
installing to /opt/app-root/src/.local/bin
  uv
  uvx
everything's installed!


In [2]:
# Step 1: Install necessary libraries (run in a cell if needed)
!uv pip install -r requirements.txt

Using Python 3.11.11 environment at: /opt/app-root
Audited 10 packages in 22ms


# Import libraries needed for this notebook

⚠️ Note. You can safely ignore the *TqdmWarning*. This is simply because this JupyterLab notebook image does not have the ipywidgets progress bar widget installed.

In [3]:
from utils import *
from pymilvus import connections, utility, Collection, CollectionSchema, FieldSchema, DataType
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

/opt/app-root/lib64/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# Initialise important variables
Initialise key variables used in this lab.  
Ideally these should have been defined in the Workbench, but for this lab we are defining them in the notebook.

In [4]:
milvus_uri="http://milvus-service.milvus.svc.cluster.local:19530"
llm_uri = "http://llama3-2-3b-predictor.llama-serving.svc.cluster.local:8080/v1/completions"

# Shakeout minio connection

In [5]:
#!/usr/bin/env python3
"""
Shake-out test for a MinIO deployment on Kubernetes.

Environment variables:
  AWS_S3_ENDPOINT        – MinIO service DNS name (e.g. minio.minio.svc.cluster.local)
  AWS_ACCESS_KEY_ID      – MinIO access key
  AWS_SECRET_ACCESS_KEY  – MinIO secret key
  AWS_DEFAULT_REGION     – Dummy value; boto3 still expects one
"""
import os
import sys

import boto3
from botocore.client import Config
from botocore.exceptions import BotoCoreError, ClientError


endpoint = os.getenv("AWS_S3_ENDPOINT", "minio.minio.svc.cluster.local")
access_key = os.getenv("AWS_ACCESS_KEY_ID", "minio")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY", "test")
region = os.getenv("AWS_DEFAULT_REGION", "us-east-1") or "us-east-1"
bucket= os.getenv("AWS_S3_BUCKET", "rag-docs") or "rag-docs"


minio_status = "🟢 OK"

try:
    s3 = boto3.client(
        "s3",
        endpoint_url=f"http://{endpoint}",
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_key,
        region_name=region,
        config=Config(signature_version="s3v4"),
    )

    resp = s3.list_buckets()
    buckets = [b["Name"] for b in resp.get("Buckets", [])]

    if buckets:
        print("🟢 Connection succeeded – buckets discovered:")
        for name in buckets:
            print(f"  • {name}")
    else:
        print("🟢 Connected but no buckets found.")

except (BotoCoreError, ClientError) as exc:
    print(f"🔴 MinIO connectivity test failed: {exc}", file=sys.stderr)
    minio_status="🔴 FAIL"

🟢 Connection succeeded – buckets discovered:
  • data
  • models
  • rag-docs


# Shakeout the Milvus connectivity


## Create the Milvus database

In [6]:
milvus_status = "🟢 OK"
# This is the name of the collection that this program will use.
collection_name = "shakeout_collection"

try:
    # Create the client object
    connections.connect(
        uri=milvus_uri,
        alias="default"
    )
    
    # Make sure we start with a clean slate by deleting the collection if it exists from a prior run.
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    print(f"Collection list: {utility.list_collections()}") 

    # Databases need a schema. In this lab the schema will consist 
    # of an identifier and a vector that contains the embedding of a text string.
    
    # Define the primary key field for unique record identification
    id_field = FieldSchema(
        name="id",
        dtype=DataType.INT64,
        is_primary=True,
        auto_id=False
    )
    
    # Specify embedding model and its output dimension
    embedding_model = "all-MiniLM-L6-v2"  # Example Hugging Face model
    embedding_dim = 384  # Embedding vector size as per the model
    
    # Define the vector field to hold embedding values
    embedding_field = FieldSchema(
        name="embedding",
        dtype=DataType.FLOAT_VECTOR,
        dim=embedding_dim
    )
    
    # Assemble collection schema combining ID and embedding fields
    schema = CollectionSchema(
        fields=[id_field, embedding_field],
        description="Milvus shakeout test",
        enable_dynamic_field=False
    )

    # Instantiate the Milvus collection using the defined schema and configuration
    collection = Collection(
        name=collection_name, 
        schema=schema, 
        using='default', 
        shards_num=2,
        consistency_level="Strong"
    )
    
    # List all collections in Milvus to confirm creation
    print(f"Collection list: {utility.list_collections()}")

    # Close the Milvus connection
    collection.release()
    utility.drop_collection(collection_name)
except:
    milvus_status="🔴 FAIL"

Collection list: []
Collection list: ['shakeout_collection']


# Download documents used in this module
This step will download a PDF document from the Git repository and save it in the Minio S3 storage in the *rag-docs* bucket.

In [7]:
# Copy all of the source documents into S3
download_status = "🟢 OK"

# Define the list of files to check
file_list = [
    "https://raw.githubusercontent.com/odh-labs/rhoai-roadshow-v2/rag-dev/docs/2-rag/data/2502.07835v1.pdf"
]

try:
    if download_source_docs( s3, bucket, file_list ) == False:
        download_status="🔴 FAIL"    
except Exception as e:
    print(f"Error occurred: {e}")
    download_status="🔴 FAIL"

INFO: download_source_docs()
INFO: validate_bucket()
🪣 Bucket 'rag-docs' exists.
⬇️ Downloading: https://raw.githubusercontent.com/odh-labs/rhoai-roadshow-v2/rag-dev/docs/2-rag/data/2502.07835v1.pdf
⬆️ Uploading: 2502.07835v1.pdf to bucket rag-docs
🟢 Uploaded '2502.07835v1.pdf' successfully.


# Validate LLM connectivity
Check that the LLM that is used by this lab is available.

In [8]:
import requests
import json

llm_status = "🟢 OK"

payload = {
    "model": "llama3-2-3b",
    "prompt": "What is the capital of Australia?\nA:",
    "max_tokens": 50,
    "temperature": 0.7,
    "stop": ["\n"]  # Prevent it from generating extra questions
}

headers = {"Content-Type": "application/json"}

try:
    response = requests.post(llm_uri, headers=headers, data=json.dumps(payload), timeout=10)
    if response.ok:
        result = response.json()
        print("✅ vLLM responded successfully:")
        print("Completion:", result["choices"][0]["text"].strip())
    else:
        print(f"⚠️ vLLM responded with status code {response.status_code}:")
        print(response.text)
        llm_status = "🔴 FAIL"
except requests.exceptions.RequestException as e:
    print(f"❌ Failed to reach vLLM at {llm_uri}: {e}")
    llm_status = "🔴 FAIL"

✅ vLLM responded successfully:
Completion: Canberra


In [9]:
print(f"Minio connectivity status: {minio_status}")
print(f"Milvus status: {milvus_status}")
print(f"File download status: {download_status}")
print(f"LLM status: {llm_status}")

print("\nIf all results are OK, return to your lab workbook for further instructions. If any have an error please inform your instructor.")

Minio connectivity status: 🟢 OK
Milvus status: 🟢 OK
File download status: 🟢 OK
LLM status: 🟢 OK

If all results are OK, return to your lab workbook for further instructions. If any have an error please inform your instructor.
